In [ ]:
# Get needed auxiliary files for colab
!git clone https://github.com/philippmatthes/diplom
%cd /content/diplom/src
!mkdir shl-dataset
!wget -nc -O shl-dataset/challenge-2020-validation.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2020/challenge-2020-validation.zip

Cloning into 'diplom'...
remote: Enumerating objects: 1375, done.
remote: Counting objects: 100% (712/712), done.
remote: Compressing objects: 100% (445/445), done.
remote: Total 1375 (delta 344), reused 582 (delta 229), pack-reused 663
Receiving objects: 100% (1375/1375), 27.92 MiB | 20.42 MiB/s, done.
Resolving deltas: 100% (720/720), done.
/content/diplom/src
--2021-08-07 09:10:53--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl-dataset.org)|37.187.125.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852446972 (5.5G) [application/zip]
Saving to: ‘shl-dataset/challenge-2019-train_torso.zip’

shl-dataset/challen 100%[===================>]   5.45G  9.25MB/s    in 9m 48s  

2021-08-07 09:20:41 (9.49 MB/s) - ‘shl-dataset/challenge-2019-train_torso.zip’ saved [5852446972/5852446972]

--2021-08-07 09:20:4

In [1]:
# Switch to src dir and select tensorflow
%cd /content/diplom/src
%tensorflow_version 2.x

/content/diplom/src


In [1]:
# Define all datasets to train our model on

from pathlib import Path

DATASET_DIR = Path('shl-dataset/challenge-2020-validation.zip')

In [2]:
from tensorflow import keras

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16864235248065555050,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3215029044
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2269128632353336532
 physical_device_desc: "device: 0, name: GeForce GTX 770, pci bus id: 0000:01:00.0, compute capability: 3.0"]

In [3]:
from tools.dataset import load_zipped_shl_dataset

from tqdm import tqdm

dataset = load_zipped_shl_dataset(DATASET_DIR, tqdm=tqdm, subdir_in_zip='validation')

Extracting shl-dataset\challenge-2020-validation.zip: 100%|██████████| 89/89 [01:56<00:00,  1.31s/it]


Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done
Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done
Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done
Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done


In [ ]:
# Check that we don't have NaNs in our dataset
assert not np.isnan(dataset.acc_mag).any()
assert not np.isnan(dataset.mag_mag).any()
assert not np.isnan(dataset.gyr_mag).any()

In [4]:
import joblib

from sklearn.preprocessing import PowerTransformer

acc_scaler = joblib.load('models/acc-scaler.joblib')
mag_scaler = joblib.load('models/mag-scaler.joblib')
gyr_scaler = joblib.load('models/gyr-scaler.joblib')

acc_mag_scaled = acc_scaler.transform(dataset.acc_mag)
mag_mag_scaled = mag_scaler.transform(dataset.mag_mag)
gyr_mag_scaled = gyr_scaler.transform(dataset.gyr_mag)

In [5]:
import numpy as np

# Prepare testing data
X = np.stack([
    acc_mag_scaled,
    mag_mag_scaled, 
    gyr_mag_scaled,
], axis=2)

del acc_mag_scaled
del mag_mag_scaled
del gyr_mag_scaled

In [6]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(dataset.labels), y=dataset.labels)
class_weights = dict(zip(np.unique(dataset.labels), class_weights)) # Keras adaption
class_weights[0] = 0 # NULL label
class_weights

{1: 0.6030878163231105,
 2: 0.688732057416268,
 3: 6.484009009009009,
 4: 1.4950664727877025,
 5: 0.8787851037851038,
 6: 1.960035403050109,
 7: 0.8249942686840899,
 8: 0.8287943344081069,
 0: 0}

In [7]:
MODEL_DIRS = [
    Path('models/shl-baseline-cnn.h5'),
    Path('models/shl-resnet.h5'),
]

for model_dir in MODEL_DIRS:
    model = keras.models.load_model(model_dir)

    score = model.evaluate(X, dataset.labels, verbose=0) 

    print(f'{model_dir} Test loss:', score[0]) 
    print(f'{model_dir} Test accuracy:', score[1])

ValueError: Unknown layer: Functional